# 🔥 GOD MODE: DreamerV3 Training on Colab

## Training Configuration
- **Target**: 1,000,000 steps (God Mode)
- **Expected Time**: 
  - Free T4: 20-30 hours (requires 2-3 session resumes)
  - Pro V100: 12-18 hours (1-2 resumes)
  - Pro+ A100: 5-7 hours (continuous)
- **Auto-checkpoint**: Every 10,000 steps
- **Auto-resume**: Continues from last checkpoint

## ✨ Updated Features (PPO Gold Trading Improvements):
- Advanced 5-component reward function (Sortino, DD penalty, etc.)
- Differential Sharpe Ratio (DSR)
- Optimized gamma=0.6 for intraday-swing trading
- Gold-optimized technical indicators (RSI 21/75-25, MACD 16/34/13, BB 13)

---

## 🚀 Instructions:
1. Upload your `drl-trading` folder to Google Drive
2. Run cells in order
3. If session disconnects, just re-run - it will resume automatically
4. Download final model when complete

In [ ]:
# ============================================================
# CELL 1: Setup & Mount Google Drive
# ============================================================

from google.colab import drive
import os

# Mount Google Drive
drive.mount('/content/drive')

# Check GPU
!nvidia-smi

print("\n✅ Google Drive mounted!")
print("📊 GPU Status shown above")

In [ ]:
# ============================================================
# CELL 2: Upload & Extract Your Project
# ============================================================

# OPTION A: If you uploaded drl-trading folder to Google Drive
# Specify the path to your folder in Google Drive:
PROJECT_PATH = '/content/drive/MyDrive/drl-trading'  # ← Adjust this path

# Copy to Colab workspace for faster training
!cp -r "$PROJECT_PATH" /content/drl-trading

# Change to project directory
os.chdir('/content/drl-trading')

print("\n✅ Project loaded!")
print(f"Working directory: {os.getcwd()}")

# OPTION B: If you have a .tar.gz or .zip file
# Uncomment these lines:
# !tar -xzf /content/drive/MyDrive/drl-trading.tar.gz -C /content/
# os.chdir('/content/drl-trading')

In [ ]:
# ============================================================
# CELL 3: Install Dependencies
# ============================================================

# Install required packages
!pip install -q torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
!pip install -q stable-baselines3
!pip install -q gymnasium
!pip install -q pandas numpy tqdm
!pip install -q yfinance  # For VIX and TIPS data

print("\n✅ All dependencies installed!")

# Verify PyTorch can see GPU
import torch
print(f"\n🔥 PyTorch GPU Available: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"   GPU Name: {torch.cuda.get_device_name(0)}")
    print(f"   CUDA Version: {torch.version.cuda}")

In [ ]:
# ============================================================
# CELL 4: Verify Data Files
# ============================================================

import os
import pandas as pd

# Check if data file exists
DATA_FILE = 'data/xauusd_1h_macro.csv'

if os.path.exists(DATA_FILE):
    df = pd.read_csv(DATA_FILE)
    print(f"✅ Data file found: {DATA_FILE}")
    print(f"   Shape: {df.shape}")
    print(f"   Columns: {list(df.columns)}")
    print(f"\n   First few rows:")
    print(df.head())
else:
    print(f"❌ ERROR: Data file not found at {DATA_FILE}")
    print("   Please ensure your data file is uploaded to Google Drive")
    print("   Expected columns: time, open, high, low, close, volume, dxy_close, spx_close, us10y_close")

In [ ]:
# ============================================================
# CELL 5: Configure Training Parameters
# ============================================================

# Training Configuration
TRAINING_STEPS = 1_000_000  # 1 Million steps for God Mode
BATCH_SIZE = 128            # Larger batch for GPU (adjust if OOM)
DEVICE = 'cuda'             # Use GPU
CHECKPOINT_FREQ = 10_000    # Save every 10k steps

# Optional: Start from existing checkpoint
RESUME_CHECKPOINT = None    # Set to checkpoint path to resume
# RESUME_CHECKPOINT = 'train/dreamer/dreamer_xauusd_step_100000.pt'

print(f"""\n🎯 Training Configuration:
   Steps: {TRAINING_STEPS:,}
   Batch Size: {BATCH_SIZE}
   Device: {DEVICE}
   Checkpoint Frequency: {CHECKPOINT_FREQ:,}
   Resume From: {RESUME_CHECKPOINT or 'Scratch'}
""")

In [ ]:
# ============================================================
# CELL 6: START TRAINING (Main Cell)
# ============================================================

# This is the main training cell - it will run for hours
# If Colab disconnects, just re-run this cell - it will resume automatically

import subprocess
import sys

print("\n" + "="*60)
print("🚀 STARTING GOD MODE TRAINING")
print("="*60 + "\n")

# Build command
cmd = [
    sys.executable,
    'train/train_dreamer.py',
    '--device', DEVICE,
    '--batch-size', str(BATCH_SIZE),
    '--steps', str(TRAINING_STEPS)
]

if RESUME_CHECKPOINT:
    cmd.extend(['--resume', RESUME_CHECKPOINT])

# Run training
try:
    subprocess.run(cmd, check=True)
    print("\n" + "="*60)
    print("✅ TRAINING COMPLETE!")
    print("="*60)
except subprocess.CalledProcessError as e:
    print(f"\n❌ Training error: {e}")
except KeyboardInterrupt:
    print("\n⚠️ Training interrupted by user")

In [ ]:
# ============================================================
# CELL 7: Check Training Progress
# ============================================================

# Run this cell anytime to check progress (while training runs)

import os
import glob

# List all checkpoints
checkpoints = sorted(glob.glob('train/dreamer/*.pt'))

print(f"\n📊 Training Progress:\n")
print(f"Total checkpoints: {len(checkpoints)}")

if checkpoints:
    print("\nLatest checkpoints:")
    for cp in checkpoints[-5:]:
        size_mb = os.path.getsize(cp) / (1024 * 1024)
        print(f"  • {os.path.basename(cp)} ({size_mb:.1f} MB)")
    
    # Estimate progress if checkpoint naming includes step number
    latest = checkpoints[-1]
    if 'step_' in latest:
        try:
            step = int(latest.split('step_')[1].split('.')[0])
            progress = (step / TRAINING_STEPS) * 100
            print(f"\n📈 Progress: {step:,} / {TRAINING_STEPS:,} steps ({progress:.1f}%)")
            
            # Estimate time remaining (rough)
            # This is approximate - actual speed varies
        except:
            pass
else:
    print("  No checkpoints yet. Training may be initializing...")

In [ ]:
# ============================================================
# CELL 8: Download Final Model
# ============================================================

# After training completes, run this to download the model

from google.colab import files
import shutil

# Create archive with all important files
!tar -czf trained_model_1m.tar.gz train/dreamer/*.pt

print("\n📦 Downloading trained model...")
files.download('trained_model_1m.tar.gz')

print("\n✅ Download complete!")
print("   Extract with: tar -xzf trained_model_1m.tar.gz")

In [ ]:
# ============================================================
# CELL 9: Backup to Google Drive
# ============================================================

# Automatically backup final model to Google Drive

import shutil
from datetime import datetime

# Create backup folder with timestamp
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
backup_path = f'/content/drive/MyDrive/drl-trading-backup-{timestamp}'

# Copy trained models
!mkdir -p "$backup_path"
!cp -r train/dreamer "$backup_path/"

print(f"\n✅ Backup saved to Google Drive:")
print(f"   {backup_path}")
print("\n💡 Your trained model is now safely backed up!")

---

## 🎯 What's Next?

After training completes:

1. **Download the model** (Cell 8)
2. **Validate on crises**: Run `python eval/crisis_validation.py`
3. **Test on demo account**: Use the live trading script
4. **Monitor performance**: Track results for 30+ days

---

## 🔥 Troubleshooting

### Session Disconnected?
- Just re-run Cell 6 - it will resume from last checkpoint

### Out of Memory?
- Reduce `BATCH_SIZE` to 64 or 32 in Cell 5
- Restart runtime and re-run

### Training Too Slow?
- Check Cell 1 - is GPU enabled?
- Consider upgrading to Colab Pro+ for A100

### Data Not Found?
- Verify path in Cell 2
- Check data file in Cell 4

---

**🏆 Welcome to God Mode Training!**
